---
title: Using R and Python in a Quarto Document
subtitle: Why debate which is better when you can use both
date: "2024-11-12"
categories: [News, Python]
code-fold: false
code-copy: hover
code-summary: "Show the code"
image: image.png
engine: jupyter
---


## A Short Introduction
I saw an interesting blog post by Nicole Renner on [combining R and Python in a Quarto document](https://nrennie.rbind.io/blog/combining-r-and-python-with-reticulate-and-quarto/) and I decided to replicate that. Quarto has been seen as the next generation of RMarkdown with support for multiple languages. In this blog post we will interchangeably use the R and Python for a simple EDA on the Palmer penguins dataset. To ensure easy interchange between both languages, we will use the [reticulate package](https://rstudio.github.io/reticulate/) by Thomasz Kalinowski, Kevin Ushey, J. J. Allaire, and Yuan Tang. 


## The Reticulate Package
The reticulate package provides a set of tool for the coordination between python and R. This means that it allows python to be used interactively within an R session. This is beautiful, because the strength of both packages can be utilized to the fullest. To know more about the reticulate package the [documentation](https://rstudio.github.io/reticulate/).


## Initial Set Up
The version of R used is `{r} R.version.string` and RStudio is `Cranberry Hibiscus Release (c8fc7aee, 2024-09-16) for Ubuntu Jammy`. To ensure interoperability between R and Python there are some things we need to put in place.  First, we need to ensure that python is properly configured to work in R studio. On the menus click on `Tools` > `Global Options` > `Python` > `find interpreter` and choose a python version of your choice. Next we have to install the reticulate package. Entering the command below to install reticulate

```{r}
#| label: install-reticulate
#| message: false
#| warning: false
#| eval: false
# R Code

install.packages("reticulate")
```


After that, load the package into your current session and that is it. The other R packages to be used for this project in R will also be loaded at once.

```{r}
#| label: load-reticulate

# R Code
library(reticulate)
library(tidyverse)
library(ggthemes)
library(latex2exp)
```

```{r}
use_python("/usr/bin/python3", required = TRUE)
py_config()
```



We can use the [pip](https://pip.pypa.io/en/stable/installation/) to also install necessary packages we'll be needing.


In [ ]:
#| label: install-py-pak
#| eval: false
#| warning: false

# Python Code
!pip install pandas 

The above code which is not evaluated/executed would install pandas. 


In [ ]:
#| label: import-python-packages
# Python Code


import pandas as pd

On running the above command, reticulate will call the `repl_python()` function which makes us interact with python in rstudio. This switches the rstudio console from `>` of R to `>>>` of python. Seeing these symbols indicates that we can execute python commands. To return to R you can enter `exit` or `quit` in the console, but you probably won't want to exit and restart python interpreter. I'll show a better way to use both program(R and Python) and using them with quarto seems to make them work seamlessly.
## EDA on Palmer Penguins

The approach to this post will be direct Python will be used for data importation and summaries and R for visualization. I'm using R for visualization because in my opinion it possesses the best visualization library. The palmerpenguins data is saved as a csv file and we'll start with a python code to read the data.


In [ ]:
#| label: import-data
# Python Code
penguins = pd.read_csv("data/penguins.csv")

Next we check the basic characteristics of the data.

In [ ]:
# Python Code
penguins.info()

The `Unnamed` column is not needed and will be dropped.


In [ ]:
# Python Code
penguins_norm = penguins.drop("Unnamed: 0", axis=1)

We can check to see the column names


In [ ]:
# Python Code
penguins_norm.columns

Next let's investigate the data for missingness

In [ ]:
# Python Code
penguins_norm.isna().sum()

Some of the columns such as sex, flipper_length_mm, and body_mass_g among others are having missing values. We can remove this and proceed with the analysis.


In [ ]:
# Python Code
penguins_clean = penguins_norm.dropna()
penguins_clean.isna().sum()

We can easily get a good description of the data.

In [ ]:
#| label: data-summary
# Python Code
penguins_clean.describe()

This has worked sweetly so far for python. Just to let us know we can replicate the processes above using R. We can do this by interacting with the python main module using the reticulate object [py](https://rstudio.github.io/reticulate/reference/py.html). This helps us interacts with all python objects created so. The syntax is straightforward. We use the `$` with `py` to access the object of interest in Python's session.

```{r}
#| label: using-r
#| warning: false
#| message: false
# R Code

penguins_r <- py$penguins |> 
  select(-`Unnamed: 0`) |> 
  drop_na()

head(penguins_r)
```


Just like `py`, we have `r` to access objects of R session within python. Instead of using `$`, the dot, (`.`) is used instead. `penguins_r` is the same as `penguins_clean`. We will do some quick summaries using the `penguins_r` to show how we get R objects into python sessions


In [ ]:
# Python Code

penguins_cleaned = r.penguins_r

penguins_cleaned.head()

A good start to our analysis would be to count the total number of penguins according to species, then according to their sex


In [ ]:
# Python Code
penguins_cleaned.groupby(["species", "sex"]).count()

Now let's make quick summaries using the data. But first we'll get all numeric data variable.


In [ ]:
# Python Code
numeric_var = penguins_cleaned.loc[:, penguins_cleaned.dtypes == "float64"].columns
numeric_var = numeric_var.drop("year")
numeric_var

Using this, we can make good summaries. 

Average body mass, bill length, bill depth, and flipper length of penguins according to species and their sexes.

In [ ]:
# Python Code
penguins_cleaned.groupby(["species", "sex"])[numeric_var].aggregate("mean").round(1)

It seems male generally have higher body mass, flipper length, bill length and depth than females for each species. It would be interesting to see if there's a significant difference in the variables between these species and sexes.

We can also make a pivot table of these metrics according and include the `island`.

In [ ]:
# Python Code
penguins_cleaned.pivot_table(values=numeric_var, index="island", columns=["species", "sex"], aggfunc=["mean", "std"], fill_value=0).round(1).unstack()

Given all that's done we can start visualizing the data. First we get our data from the python environment.

```{r}
# R Code
penguins_tbl <- py$penguins_cleaned
```


### Body Mass vs Flipper Length

```{r}
# R Code
ggplot(penguins_tbl, aes(flipper_length_mm, body_mass_g, col = species)) +
  geom_point() +
  geom_smooth(
    method = "lm",
    se = FALSE,
    linewidth = .5
  ) +
  scale_fill_colorblind() +
  labs(
    x = TeX(r"(Flipper Length ($mm$))"),
    y = TeX(r"(Body Mass ($g$))"),
    title = TeX("Body Mass vs Flipper Length", bold = TRUE),
    col = TeX("Species", bold = TRUE)
  ) + 
  theme_fivethirtyeight()
```


### Key Metrics Distribution

```{r}

# R Code

penguins_long <- penguins_tbl |> 
  pivot_longer(
    cols = bill_length_mm:body_mass_g
  ) |> 
  mutate(
    name = str_replace_all(name, "_", " "),
    name = str_to_title(name),
    name = str_replace_all(name, "Mm", "(mm)"),
    name = str_replace(name, "G", "(g)")
  )

penguins_long |> 
  ggplot(aes(factor(year), value, fill = species)) +
  geom_boxplot(
    position = "dodge",
    outlier.colour = "red",
    linewidth = 1
  ) +
  geom_violin(
    position = "dodge",
    alpha = .2
  ) +
  facet_wrap(~name, scales = "free") +
  labs(
    x = "Year",
    title = TeX(r"(\textbf{Distribution of Key Characteristics from 2007 to 2009})")
  ) +
  scale_fill_calc() +
  theme_fivethirtyeight() 
```


## Conclusion
This project highlights one of the strength of using quarto and RMarkdown, and that's their compatibility with other languages. Here, I used both R and Python in the same project and switched the use of both languages. This is possible using the [reticulate](https://rstudio.github.io/reticulate/index.html) package in R, and there's a similar package to allow easy integration of R in python with the [rpy2](https://pypi.org/project/rpy2/) package.